<a href="https://colab.research.google.com/github/yota-nagai/combinatorial-optimization-problems/blob/main/08_%E6%95%B4%E6%95%B0%E9%95%B7%E3%82%B8%E3%83%A7%E3%83%96%E3%82%B9%E3%82%B1%E3%82%B8%E3%83%A5%E3%83%BC%E3%83%AA%E3%83%B3%E3%82%B0%E5%95%8F%E9%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q amplify
from amplify import VariableGenerator, FixstarsClient, solve, Solver
from datetime import timedelta

client = FixstarsClient()
client.token = "******************"  #アクセストークンを入力
client.parameters.timeout = timedelta(milliseconds=2000)  # タイムアウトは 2000 ms


#################### 整数長ジョブスケジューリング問題 ####################

"""
うまくいかない
f1のペナルティ係数を大きくしてもアサインされてないジョブが出てくる
f2がおかしそう
各コンピュータの処理時間を全体の平均時間に近づけたい
"""

N=int(input())  #ジョブの個数
L=list(map(int,input().split()))  #各ジョブの長さ
C=int(input())  #コンピュータの数
M=sum(L)//C  #各コンピュータにアサインされるジョブ長と最大ジョブ長（コンピュータ0に固定）にどの程度差があるか

#バイナリ変数の生成
gen=VariableGenerator()
q=gen.array("Binary",shape=(N,C))  #ジョブiがコンピュータcにアサインされているか
q2=gen.array("Binary",shape=(M+1,C)) # M_1-M_c=nを満たすかどうか

#制約項①　各ジョブは1台のコンピュータにアサイン
f1=0
for i in range(N):
  cnt=0
  for c in range(C):
    cnt+=q[i][c]
  f1+=(1-cnt)**2

#制約項②　処理時間最大のコンピュータをコンピュータ0に設定
f2=0
for c in range(C):
  cnt=0
  for n in range(M+1):
    cnt+=n*q2[n][c]
  for i in range(N):
    cnt+=L[i]*(q[i][c]-q[i][0])
  f2+=cnt**2

#目的関数項　一番長くかかるコンピュータの処理時間を最小化したい
f3=0
for i in range(N):
  f3+=L[i]*q[i][0]

#ペナルティ係数を設定
A,B=map(int,input().split())
f=A*(f1+f2)+B*f3


#ソルバー実行、最適解の取得
result=solve(f,client)
objective=result.best.objective
values=list(result.best.values.values())  # qの値（N*C個）と q2の値（(M+1)*C）が順に並んでる
values_q=values[:N+C]
values_q2=values[N*C:]

#エネルギー関数を最小にするアサイン方法を1つ出力
print(f"エネルギー関数の最小値 = {objective}")
assigned=[0]*C
for pos in range(len(values_q)):
  if values_q[pos]==1:
    assigned[pos%C]+=L[pos//C]

print(f"各コンピュータにアサインされたジョブ長 : {assigned} , ジョブ長最大値 : {max(assigned)}")